Pre-Processing Code Adapted From the Following Repo:
https://github.com/jfzhang95/pytorch-video-recognition

Pre-Trained R3D Model Taken From The Following Repo:
https://github.com/kenshohara/3D-ResNets-PyTorch

Code below specifying the 3D Resnet model taken from the aformentioned repo

In [0]:
import math
from functools import partial
import torch
import torch.nn as nn
import torch.nn.functional as F


def get_inplanes():
    return [64, 128, 256, 512]

'''
3D conv layers
'''
def conv3x3x3(in_planes, out_planes, stride=1):
    return nn.Conv3d(in_planes,
                     out_planes,
                     kernel_size=3,
                     stride=stride,
                     padding=1,
                     bias=False)


def conv1x1x1(in_planes, out_planes, stride=1):
    return nn.Conv3d(in_planes,
                     out_planes,
                     kernel_size=1,
                     stride=stride,
                     bias=False)


'''
Resnet Block
'''
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1, downsample=None):
        super().__init__()

        self.conv1 = conv3x3x3(in_planes, planes, stride)
        self.bn1 = nn.BatchNorm3d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3x3(planes, planes)
        self.bn2 = nn.BatchNorm3d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out

'''
Full 3D Resnet
'''

class ResNet(nn.Module):

    def __init__(self,
                 block,
                 layers,
                 block_inplanes,
                 n_input_channels=3,
                 conv1_t_size=7,
                 conv1_t_stride=1,
                 no_max_pool=False,
                 shortcut_type='B',
                 widen_factor=1.0,
                 n_classes=1039):
        super().__init__()

        block_inplanes = [int(x * widen_factor) for x in block_inplanes]

        self.in_planes = block_inplanes[0]
        self.no_max_pool = no_max_pool

        self.conv1 = nn.Conv3d(n_input_channels,
                               self.in_planes,
                               kernel_size=(conv1_t_size, 7, 7),
                               stride=(conv1_t_stride, 2, 2),
                               padding=(conv1_t_size // 2, 3, 3),
                               bias=False)
        self.bn1 = nn.BatchNorm3d(self.in_planes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool3d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, block_inplanes[0], layers[0],
                                       shortcut_type)
        self.layer2 = self._make_layer(block,
                                       block_inplanes[1],
                                       layers[1],
                                       shortcut_type,
                                       stride=2)
        self.layer3 = self._make_layer(block,
                                       block_inplanes[2],
                                       layers[2],
                                       shortcut_type,
                                       stride=2)
        self.layer4 = self._make_layer(block,
                                       block_inplanes[3],
                                       layers[3],
                                       shortcut_type,
                                       stride=2)

        self.avgpool = nn.AdaptiveAvgPool3d((1, 1, 1))
        self.fc = nn.Linear(block_inplanes[3] * block.expansion, n_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv3d):
                nn.init.kaiming_normal_(m.weight,
                                        mode='fan_out',
                                        nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm3d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def _downsample_basic_block(self, x, planes, stride):
        out = F.avg_pool3d(x, kernel_size=1, stride=stride)
        zero_pads = torch.zeros(out.size(0), planes - out.size(1), out.size(2),
                                out.size(3), out.size(4))
        if isinstance(out.data, torch.cuda.FloatTensor):
            zero_pads = zero_pads.cuda()

        out = torch.cat([out.data, zero_pads], dim=1)

        return out

    def _make_layer(self, block, planes, blocks, shortcut_type, stride=1):
        downsample = None
        if stride != 1 or self.in_planes != planes * block.expansion:
            if shortcut_type == 'A':
                downsample = partial(self._downsample_basic_block,
                                     planes=planes * block.expansion,
                                     stride=stride)
            else:
                downsample = nn.Sequential(
                    conv1x1x1(self.in_planes, planes * block.expansion, stride),
                    nn.BatchNorm3d(planes * block.expansion))

        layers = []
        layers.append(
            block(in_planes=self.in_planes,
                  planes=planes,
                  stride=stride,
                  downsample=downsample))
        self.in_planes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.in_planes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        if not self.no_max_pool:
            x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)

        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

'''
Function to create model
'''
def generate(model_depth, **kwargs):
    assert model_depth in [10, 18, 34, 50, 101, 152, 200]

    if model_depth == 10:
        model = ResNet(BasicBlock, [1, 1, 1, 1], get_inplanes(), **kwargs)
    elif model_depth == 18:
        model = ResNet(BasicBlock, [2, 2, 2, 2], get_inplanes(), **kwargs)
    elif model_depth == 34:
        model = ResNet(BasicBlock, [3, 4, 6, 3], get_inplanes(), **kwargs)

    return model

'''
Function to load pretrained model
'''
def load_pretrained_model(model, pretrain_path, model_name, n_finetune_classes):
    if pretrain_path:
        print('loading pretrained model {}'.format(pretrain_path))
        pretrain = torch.load(pretrain_path, map_location='cpu')

        model.load_state_dict(pretrain['state_dict'])
        tmp_model = model
        if model_name == 'densenet':
            tmp_model.classifier = nn.Linear(tmp_model.classifier.in_features,
                                             n_finetune_classes)
        else:
            tmp_model.fc = nn.Linear(tmp_model.fc.in_features,
                                     n_finetune_classes)

    return model


**Training Loop**

In [0]:

import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torch.autograd import Variable


model_depth = 34
j = generate(model_depth)
model = load_pretrained_model(j, 'r3d34_KM_200ep.pth', 'resnet', 51)

for param in model.parameters():
    param.requires_grad = False

model.fc = nn.Linear(512, 51)
model.train()

'''
Hyperparameters
'''
batch_size = 32
num_epochs = 20
weight_decay = 1e-5
learning_rate = 5e-4


loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.fc.parameters(), lr=5e-4, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[15], gamma=0.1)

model.cuda()
loss_function.cuda()

'''
Preprocessing and dataloaders
'''

train_dataloader = DataLoader(VideoDataset(dataset='hmdb51', split='train',clip_len=16), batch_size=32, shuffle=True, num_workers=4)
val_dataloader   = DataLoader(VideoDataset(dataset='hmdb51', split='val',  clip_len=16), batch_size=32, num_workers=4)
test_dataloader  = DataLoader(VideoDataset(dataset='hmdb51', split='test', clip_len=16), batch_size=32, num_workers=4)

'''
Initializing ist of training performance metrics
'''

training_loss = []
training_acc = []
validation_loss = []
validation_acc = []


'''
Training_Loop
'''
for z in range(num_epochs):
    for i, (inputs, labels) in enumerate(train_dataloader):
        inputs = inputs.cuda()
        labels = labels.cuda()
        outputs = model(inputs)

        probs = nn.Softmax(dim=1)(outputs)
        preds = torch.max(probs, 1)[1]
        loss = loss_function(outputs, labels)
        accuracy = torch.sum(preds == labels.data).item()

        print("Epoch:",z, "Loss:",loss.item(), "Accuracy:", accuracy/32)
        training_loss.append(loss.item())
        training_acc.append(accuracy/32)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if i % 40 == 0 and i != 0:
            with torch.no_grad():
              total_val_loss = 0
              total_val_acc = 0
              model.eval()

            m = 0
            for j, (inputs, labels) in enumerate(val_dataloader):
                inputs = inputs.cuda()
                labels = labels.cuda()

                outputs = model(inputs)
                val_probs = nn.Softmax(dim=1)(outputs)
                val_loss = loss_function(outputs, labels)
                val_preds = torch.max(val_probs, 1)[1]
                total_val_acc += torch.sum(val_preds == labels.data).item()

                total_val_loss += val_loss.item()
                m += 1

            print("Validation Loss:", total_val_loss/m, "acc:",total_val_acc/32/m)
            validation_loss.append(total_val_loss/m)
            validation_acc.append(total_val_acc/32/m)
            model.train()
            
    scheduler.step()

    '''
    Saving model and performance metrics
    '''
    torch.save(model.state_dict(), 'R3D_big3.pt')
    np.save('Training_Loss_6', np.array(training_loss))
    np.save('Training_Accuracy_6', np.array(training_acc))
    np.save('Val_Loss_6', np.array(validation_loss))
    np.save('Val_Accuracy_6', np.array(validation_acc))

**Test Set Performance**


In [0]:

m = 0
with torch.no_grad():
              total_test_loss = 0
              total_test_acc = 0
              model.eval()
loss_function = nn.CrossEntropyLoss()
for j, (inputs, labels) in enumerate(test_dataloader):
    inputs = inputs.cuda()
    labels = labels.cuda()

    outputs = model(inputs)
    test_probs = nn.Softmax(dim=1)(outputs)
    test_loss = loss_function(outputs, labels)
    preds = torch.max(test_probs, 1)[1]
    print(test_loss)

    total_test_loss += test_loss.item()
    total_test_acc += (torch.sum(preds == labels.data).item() / len(labels))
    m += 1

print('Test Loss:', total_test_loss/m)
print('Test Accuracy:', total_test_acc / m)